In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
rock = np.load("janken/rock.npy")
scissors = np.load("janken/scissors.npy")
paper = np.load("janken/paper.npy")

rock.shape, scissors.shape, paper.shape

((244, 21, 3), (338, 21, 3), (283, 21, 3))

In [3]:
def plot_landmarks(landmarks):
    landmarks = np.array([l - l.mean(axis=0) for l in landmarks])
    fig = plt.figure(figsize=(6, 4))
    gs = fig.add_gridspec(12, 1)
    ax = fig.add_subplot(gs[:-2, :], projection="3d")
    ax_slider = fig.add_subplot(gs[-1, :])
    slider = Slider(ax_slider, "index", 0, len(landmarks), valinit=0, valstep=1)

    scatters = [ax.scatter3D(*landmarks[idx, :, :].T) for idx in range(len(landmarks))]
    for s in scatters:
        s.set_visible(False)

    def update(v):
        for s in [s for s in scatters if s.get_visible()]:
            s.set_visible(False)
        idx = int(slider.val)
        scatters[idx].set_visible(True)

    slider.on_changed(update)
    update(None)

    ax.set(xlim=(-0.2, 0.2), ylim=(-0.4, 0.4), zlim=(0, 1), xlabel="x", ylabel="y", zlabel="z")
    ax.view_init(elev=95, azim=90)
    fig.tight_layout()


# plot_landmarks(rock)
# plot_landmarks(paper)
plot_landmarks(scissors)
# plot_landmarks(nothing)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
hands = np.array([h - h.mean(axis=0) for h in np.concatenate([rock, scissors, paper])])
hands = np.repeat(hands, 2, axis=0)
hands[: hands.shape[0] // 2, 0] = -hands[: hands.shape[0] // 2, 0]

labels = np.array([0] * len(rock) + [1] * len(scissors) + [2] * len(paper))
labels = np.repeat(labels, 2, axis=0)

hands_train, hands_test, labels_train, labels_test = train_test_split(hands, labels)

hands_train.shape

(1297, 21, 3)

In [5]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(21, 3)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(3, activation="softmax"),
    ]
)
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
model.fit(hands_train, labels_train, epochs=100, verbose=0)
model.evaluate(hands_test, labels_test)
# model.save("janken_tf")

14/14 [==============================] - 0s 657us/step - loss: 9.9387e-05 - accuracy: 1.0000


[9.938684524968266e-05, 1.0]

In [6]:
hands = np.array([h - h.mean(axis=0) for h in rock])

model = tf.keras.models.load_model("janken_tf")
model.predict(hands[:5])

array([[9.9986219e-01, 7.5805692e-06, 1.3021659e-04],
       [9.9988067e-01, 7.0494070e-06, 1.1232400e-04],
       [9.9994576e-01, 5.6556792e-06, 4.8591493e-05],
       [9.9994338e-01, 5.0090939e-06, 5.1634815e-05],
       [9.9992275e-01, 5.8962951e-06, 7.1400282e-05]], dtype=float32)